In [48]:
import pandas as pd

In [49]:
train = pd.read_csv("train.csv")

In [50]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### firstly,fill all the null spaces with a space

In [51]:
train = train.fillna("")

train["total"] = train["title"]+" "+train["author"]+" "+train["text"]

train.head()

,id,title,author,text,label,total
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


 ## pre-processing / cleaning the data

In [52]:
import nltk

nltk.download("punkt")

from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [54]:
nltk.download("stopwords")

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


### Tokenization: Word tokenization is the process of splitting a large sample of text into words

In [55]:
from nltk.tokenize import TweetTokenizer 

tknzr = TweetTokenizer()

word_data = "It originated from the idea that there are readers who prefer learning new skills from the comforts of their drawing rooms"

nltk_tokens = tknzr.tokenize(word_data)

print(nltk_tokens)

['It', 'originated', 'from', 'the', 'idea', 'that', 'there', 'are', 'readers', 'who', 'prefer', 'learning', 'new', 'skills', 'from', 'the', 'comforts', 'of', 'their', 'drawing', 'rooms']


In [56]:
lemmatizer = WordNetLemmatizer()

### The code below is for lemmatization for our test data which excludes stopwords at the same time.

In [58]:
nltk.download("wordnet")
import re

from nltk.tokenize import sent_tokenize, word_tokenize 

for index, row in train.iterrows():
    
    filter_sentence = ''

    sentence = row['total']
    
    # Cleaning the sentence with regex
    sentence = re.sub(r'[^\w\s]', '', sentence)

    
    # Tokenization
    
    words = nltk.word_tokenize(sentence)
    
    # Stopwords removal
    
    words = [w for w in words if not w in stop_words]
    # Lemmatization
    
    for words in words:
        
        filter_sentence = filter_sentence  + ' ' +str(lemmatizer.lemmatize(words)).lower()
        
    train.loc[index, 'total'] = filter_sentence
    
train = train[['total', 'label']]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [59]:
train.head()

,total,label
0,house dem aide we didnt even see comeys lette...,1
1,flynn hillary clinton big woman campus breitb...,0
2,why truth might get you fired consortiumnewsc...,1
3,15 civilians killed in single us airstrike ha...,1
4,iranian woman jailed fictional unpublished st...,1


In [60]:
X_train = train['total']

Y_train = train['label']

In [61]:
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

count_vectorizer = CountVectorizer()

count_vectorizer.fit_transform(X_train)

freq_term_matrix = count_vectorizer.transform(X_train)

tfidf = TfidfTransformer(norm = "l2")

tfidf.fit(freq_term_matrix)

tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix,Y_train, random_state=0)

## logistic regression

In [68]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

Accuracy_logistic_regression = logreg.score(X_test, y_test)



## naive bayes

In [69]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()

NB.fit(X_train, y_train)

Accuracy_naive_bayes = NB.score(X_test, y_test)

## Decision tree classifier

In [70]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

Accuracy_decision_tree = clf.score(X_test, y_test)

In [73]:
print(Accuracy_logistic_regression)

print(Accuracy_naive_bayes)

print(Accuracy_decision_tree)

0.9615384615384616
0.8442307692307692
0.9709615384615384


## As we can see, the decision tree classifier performed the best on the train set and gave an accuracy of 97%.